# Intermediate Project Notes: Link Predictions
> Yixuan Liu, 2024/10/16
> 
> Prelim contents for final project

## 1. Introduction of link prediction
### Basic idea

- Goal: given a graph $G(V,E)$ predict future/missing links between nodes
- General approach: The prediction process often involves calculating a **similarity score** between two nodes, where a higher score indicates a higher likelihood of a future or missing link. The similarity can be based on network topology, node attributes, or learned node embeddings.
- Applications: friendship connections, recommender systems, drug discovery, PPI, etc

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41598-019-57304-y/MediaObjects/41598_2019_57304_Fig1_HTML.png?as=webp" alt="Graph Illustration" width="600"/>


### Measuring performance
- Accuracy, Precision, Recall, and F1-Score
- AUC-ROC, AUC-PR
- MRR (mean reciprocal rank), Hit Rate: Focus on ranking performance, useful for algorithms that produce a ranked list of link predictions.
    - MRR: how high the true positive links are ranked compared to the false positives. It is often used in algorithms that output a ranked list of link predictions. $Q$ is the set of all queries (node pairs). And $rank_i$ the position of the first relevant link for the $i-th$ query.
      $$\operatorname{MRR}=\frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{\operatorname{rank}_i}$$
- Kendall’s Tau, MAP (Mean Average Precision): Evaluate the quality of rankings generated by the algorithm.


***
## 2. Topological predictors
Simple functions of the observed network topology. Mixure/stacking of these predictors could be found in [Gasemian et al,2020](https://www.pnas.org/doi/10.1073/pnas.1914950117). (or also look the review paper to see there is more)

### Global predictors
Quantify various network-level statistics and are inherited by each pair of nodes i,j that is a candidate missing link. Their primary utility is to provide global context to other predictors under supervised learning.
#### Example: # nodes, # edges, average degree, variance of the degree distribution, network diameter

### Pairwise predictors
These predictors are functions of the joint topological properties of the pair of nodes i, j being considered.

#### Example 1: Common Neighbours
The number of common neighbors between node *i* and node *j*.

$$CN(i, j) = | \Gamma(i) \cap \Gamma(j) |$$

Where:
- $\Gamma(i), \Gamma(j)$ is the set of neighbours of node *i*, noe *j*

#### Example 2: Jaccard Index
The Jaccard index measures the similarity between two sets by comparing the size of their intersection and union.

$$
J(i, j) = \frac{| \Gamma(i) \cap \Gamma(j) |}{| \Gamma(i) \cup \Gamma(j) |}
$$

#### Example 3: Adamic-Adar Index
The Adamic-Adar index assigns higher importance to less-connected neighbours.

$$AA(i, j) = \sum_{k \in \Gamma(i) \cap \Gamma(j)} \frac{1}{\log |\Gamma(k)|}$$

Where:
- *k* is a common neighbour of nodes *i* and *j*
- $\Gamma(k)$ is the degree (number of neighbours) of node *k*

### Node based predictors
These predictors are functions of the independent topological properties of the individual nodes $i$ and $j$, and thus produce a **pair of predictor values**. the particular function that converts the pair of node-based predictors into a score is learned within the supervised framework (e.g. Euclidean distance, cosine similarity).
#### Examples: degree, local clustering coeffcient, eigenvector centrality, PageRank, Katz centrality

***
## 3. Model-based predictors

### likelihood predictors
- Maximizing likelihood to the parametric model $\operatorname{Pr}(i \rightarrow j \mid \theta)$ by decomposing the network under a probabilistic generative model of network structure.
#### Example: Stochastic block model ([Newman and Reinert, 2016](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.117.078301))

### optimization predictors
- Score each pair $i, j$ according to whether adding them would increase a corresponding (non-probabilistic) objective function, like in the Map Equation or the modularity function.
#### Example: Modularity, Girvan-Newman ([Newman and Girvan, 2004](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.69.026113))

*** 
## 4. Embedding-based approaches
Find the closest node pairs based on proximity in the embedded space. (Let's not go too deep, could use some Python packages like **stellargraph** but it would be better with **PyTorch** environment)
### Random walk-based
Idea: nodes frequently appearing together in random walks should have similar embedding. Treating walks as sequences of nodes.
$$\max _{\mathbf{Z}} \sum_{(u, v) \in E} \log \sigma\left(\mathbf{Z}_u^{\top} \mathbf{Z}_v\right)+\sum_{\left(u^{\prime}, v^{\prime}\right) \notin E} \log \left(1-\sigma\left(\mathbf{Z}_{u^{\prime}}^{\top} \mathbf{Z}_{v^{\prime}}\right)\right)$$
#### Example: DeepWalk ([Perrozi et al., 2014](https://dl.acm.org/doi/10.1145/2623330.2623732)), node2vec ([Grover and Jeskovec, 2016](https://dl.acm.org/doi/10.1145/2939672.2939754))

### Graph Neural Networks (GNNs)
Idea: node aggregates feature information from its neighbors, passes it through a neural network, and updates its representation.
$$\mathcal{L}=\sum_{(u, v) \in E} \operatorname{Loss}(1, s(u, v))+\sum_{\left(u^{\prime}, v^{\prime}\right) \notin E} \operatorname{Loss}\left(0, s\left(u^{\prime}, v^{\prime}\right)\right)$$
#### Example: GAT ([Velicˇkovic ́ et al., 2018](https://arxiv.org/pdf/1710.10903)), GCN ([Kipf et al, 2016](https://arxiv.org/abs/1609.02907)), GraphSAGE ([Hamilton et al, 2017](https://arxiv.org/abs/1706.02216))

## 5. Case study: 
- Some baseline datasets, but some more decided by node features, some more about network topology: CORA, or [OGB](https://ogb.stanford.edu/docs/leader_linkprop/#ogbl-ddi) ?. Compare several methods and their performance.
- Data set might be: drug-drug interaction, collboration.

## 6. Discussions:

### Homophily vs. Heterophily (bipartite networks, Laszlo lovac, similarity complementary implementation)

- Homophily suggests that similar nodes are more likely to connect, whereas heterophily implies connections between dissimilar nodes.
- Homophily-based methods (like common neighbors or Jaccard) work well for networks where similar nodes are connected, but they struggle in heterophilic networks.
- For networks formed through heterophily (e.g., connections between dissimilar or complementary nodes), we might need different predictors or additional features (such as node attributes) to account for these dissimilarities.
Different Node and Edge Features:

### Node and link features
- Many link prediction methods primarily use topology for prediction, but in real-world scenarios, we also have node features (e.g., user interests in social networks) and edge features (e.g., interaction strength, type of connection).
- Model-based approaches or embedding techniques that can incorporate both node features and edge features (e.g., GraphSAGE, GCN) are needed to capture these richer relationships.

### Homogeneous and heterogeneous graph
- In heterogeneous graphs, different types of nodes and edges exist (e.g., user-product networks, author-paper-venue networks).
- Standard link prediction algorithms may not perform well in such settings as they assume homogeneous networks (i.e., all nodes and edges are of the same type).
- More advanced methods like heterogeneous graph neural networks (HGNNs) are required to handle the complexity of these diverse relationships.

## Comments from @Brennan and @Alyssa
- Objective functions, then look at it, why it is better
- Why would we have more methods of GNN? Which does it suit
- Karate club example: delete some edges
- Bipartite networks, Laszlo lovac, similarity complementary implementation
- Gasemian et al,2020. (or also look the review paper to see there is more)